In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/casas.csv')

In [3]:
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [7]:
X = df.drop('preco', axis=1)
y = df['preco'].copy()

In [9]:
X.head()

,tamanho,ano,garagem
0,159.0,2003,2
1,117.0,1976,2
2,166.0,2001,2
3,160.0,1915,3
4,204.0,2000,3


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [12]:
X_train.shape

(1022, 3)

In [13]:
X_test.shape

(438, 3)

In [39]:
import mlflow

In [40]:
mlflow.set_experiment('house-prices-eda')

2023/05/13 18:13:32 INFO mlflow.tracking.fluent: Experiment with name 'house-prices-eda' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/rotiv/Documentos/projetos/mlflow-test/mlflow/notebooks/mlruns/547063522674158863', creation_time=1684012412856, experiment_id='547063522674158863', last_update_time=1684012412856, lifecycle_stage='active', name='house-prices-eda', tags={}>

In [15]:
# Linear Regression

In [41]:
mlflow.start_run()

<ActiveRun: >

In [16]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [42]:
mlflow.sklearn.log_model(lr, 'lr')

/home/rotiv/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [24]:
lr_predicted = lr.predict(X_test)

In [19]:
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
import math
mse = mean_squared_error(y_test, lr_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, lr_predicted)
mlflow.log_metric('mse', mse)
mlflow.log_metric('rmse', rmse)
mlflow.log_metric('r2', r2)

In [28]:
rmse

45592.39978251848

In [29]:
mse

2078666917.9289908

In [46]:
mlflow.end_run()

In [47]:
!pip install xgboost
from xgboost import XGBRFRegressor, XGBRegressor

In [50]:
xgb_params = {
    'learning_rate': 0.2,
    'n_estimators': 50,
    'random_state': 42
}

with mlflow.start_run():
    xgb = XGBRFRegressor(**xgb_params)
    xgb.fit(X_train, y_train)
    mlflow.xgboost.log_model(xgb, 'xgboost')
    xgb_predicted = xgb.predict(X_test)
    mse = mean_squared_error(y_test, xgb_predicted)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_test, xgb_predicted)
    mlflow.log_metric('mse', mse)
    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('r2', r2)

In [38]:
mse

1319023470.5223782

In [52]:
mlflow.get_experiment_by_name('house-prices-eda')

<Experiment: artifact_location='file:///home/rotiv/Documentos/projetos/mlflow-test/mlflow/notebooks/mlruns/547063522674158863', creation_time=1684012412856, experiment_id='547063522674158863', last_update_time=1684012412856, lifecycle_stage='active', name='house-prices-eda', tags={}>

In [54]:
mlflow.list_run_infos('1')

AttributeError: module 'mlflow' has no attribute 'list_run_infos'